##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Это краткое руководство использует [Keras](https://www.tensorflow.org/guide/keras/overview) для:

1. Загрузки готового набора данных.
1. Создания модели машинного обучения нейронной сети, которая классифицирует изображения.
2. Обучения этой нейросети.
3. Оценки точности модели.

## Установка TensorFlow
 Сначала импортируйте TensorFlow в свою программу:

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.8.0


## Загрузка набора данных

Загрузите и подготовьте [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Преобразуйте данные выборки из целых чисел в числа с плавающей запятой:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Построение модели машинного обучения

Создайте модель `tf.keras.Sequential` путем наложения слоев.

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Для каждого примера модель возвращает вектор [logits](https://developers.google.com/machine-learning/glossary#logits) или [log-odds](https://developers.google.com/machine-learning/glossary#log-odds), по одному для каждого класса.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.24573854,  0.5207895 , -0.2452746 , -0.9040776 , -0.07729617,
        -0.7325078 , -0.07548641,  0.25027874, -0.14437239, -0.39478615]],
      dtype=float32)

Функция `tf.nn.softmax` преобразует эти логиты в *вероятности* для каждого класса:

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.13738103, 0.18087555, 0.08407802, 0.04350788, 0.09945682,
        0.05165112, 0.09963698, 0.13800618, 0.09300445, 0.07240198]],
      dtype=float32)

 **Примечание**. Можно встроить функцию tf.nn.softmax в функцию активации для последнего уровня сети. Хотя это может сделать выходные данные модели более интерпретируемыми, этот подход не рекомендуется, поскольку невозможно обеспечить точный и численно стабильный расчет потерь для всех моделей при использовании выходных данных softmax.

Определите функцию потерь для обучения с использованием `losses.SparseCategoricalCrossentropy`, которая принимает вектор логитов и индекс `True` и возвращает скалярную потерю для каждого примера.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

 Эта потеря равна отрицательной логарифмической вероятности истинного класса: потеря равна нулю, если модель уверена в правильном классе.

Эта необученная модель дает вероятности, близкие к случайным (1/10 для каждого класса), поэтому начальная потеря должна быть близка к `-tf.math.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.9632435

Перед началом обучения настройте и скомпилируйте модель с помощью `Model.compile`. Установите класс [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) на `adam`, установите `loss` на функцию `loss_fn`, которую вы определили ранее, и укажите метрику, которая будет оцениваться для модели, установив параметр `metrics` на `accuracy`.

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Обучение и оценка модели

Используйте метод `Model.fit` , чтобы настроить параметры модели и минимизировать потери:

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2960 - accuracy: 0.9139
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1440 - accuracy: 0.9572
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1057 - accuracy: 0.9680
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0855 - accuracy: 0.9738
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0749 - accuracy: 0.9769


Метод `Model.evaluate` проверяет производительность моделей, обычно с помощью "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" или "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0732 - accuracy: 0.9773 - 281ms/epoch - 898us/step


[0.07319818437099457, 0.9772999882698059]

Классификатор изображений теперь обучен с точностью ~ 98% на этом наборе данных.

 Если вы хотите, чтобы ваша модель возвращала вероятность, вы можете обернуть обученную модель и прикрепить к ней softmax:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [27]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.1373385e-06, 5.6124894e-10, 2.5453577e-05, 6.5359229e-04,
        3.6497000e-10, 6.4294937e-08, 2.1446948e-12, 9.9929547e-01,
        8.9937475e-07, 2.2247061e-05],
       [1.9881130e-08, 5.4412565e-05, 9.9993217e-01, 1.2991453e-05,
        4.9580383e-15, 2.7338547e-09, 2.1776827e-09, 1.3347739e-15,
        5.0589443e-07, 7.0108544e-14],
       [2.0205870e-07, 9.9805033e-01, 4.1441637e-04, 7.0568152e-05,
        5.7517769e-05, 8.2882076e-07, 1.7710190e-05, 5.0009234e-04,
        8.8169612e-04, 6.6739949e-06],
       [9.9976474e-01, 7.1328756e-09, 1.7999471e-04, 2.0789768e-07,
        3.2166113e-06, 3.3193348e-06, 2.2607754e-05, 1.9719679e-05,
        1.8173482e-07, 6.0421999e-06],
       [7.5784760e-06, 9.2294661e-10, 2.6378289e-06, 3.0786712e-08,
        9.9887329e-01, 5.2414754e-07, 3.9364190e-06, 1.4219465e-04,
        2.0349191e-07, 9.6958299e-04]], dtype=float32)>

In [34]:
model.predict(x_test[:5]).argmax(axis=1)

array([7, 2, 1, 0, 4], dtype=int64)